# Сравнение open-source солверов и их применение в ритейле

<!-- 
Оптимизация или как ее еще называют математическое программирование - это, пожалуй, один из ключевых разделов теоретической и прикладной математики, который находит свое место практически во всех аспектах нашей жизни. Действительно, каждый человек, не задумываясь об этом, практически каждый день решает оптимизационные задачи. За примерами далеко ходить не надо - все мы часто ходим в магазин или совершаем покупки в онлайн, соответственно, мы как правило хотим минимизировать наши расходы для приобретения необходимых товаров или же при ограниченном бюджете приобрести как можно больше товаров согласно приоритетам. Или же мы хотим распределить свое время так, что бы это было максимально эффективно с какой-то точки зрения. Также оптимизация используется во всех современных алгоритмах ML для обучения, подбора параметром алгоритмов. Как правило ML модели или подобного рода алгоритмы позволяют оценивать эффекты от какого либо воздействия, поэтому может возникнуть задача оптмального управления, где методы оптимизации будут чрезвычайно полезны или даже необходимы для принятия решений.

Формулировка оптимизационной задачи может быть нетривиальным процессом, часто требующие определенной экспертизы, в том числе и в предметной области - что хотим максимизировать/минимизировать, какие условия надо соблюсти и т.п. также надо понимать, а какие есть инструменты для решения задачи. На первых порах, когда проект только зарождается (MVP) нередко используется первый попавшийся солвер, нам на практике встречались даже реализации в Excel, что для MVP-решения является нормальным подходом. Однако при дальнейшем масштабировании задачи, время поиска решения может сильно возрасти, и чтобы уложиться в приемлемое расчётное время, необходимо подбирать другой солвер. В этой статье мы рассмотрим несколько открытых солверов, сравним их возможности и скорость работы на модельной задаче ценообразования.
 -->
 
Привет, Habr!
На связи отдел аналитики данных X5 TECH.

Сегодня мы поговорим про очень интересный раздел прикладной математики — оптимизацию.

Ежедневно практически каждый человек решает оптимизационные задачи даже не задумываясь об этом. 
Например, мы, как правило, хотим минимизировать наши расходы для приобретения необходимых товаров, но чтобы эти товары были максимально полезны. Полезность здесь у каждого своя — для одних она определяется количеством растительных жиров, для других — наличием привычных товаров, и тд.
Другой пример — мы хотим распределить свой маршрут в отпуске так, что посетить всё, что запланировали с минимальными временными затратами на дорогу, и не забыть почиллить на пляже. 

Оптимизация используется во всех современных алгоритмах ML для обучения, подбора параметром алгоритмов.
Как правило, ML модели или подобного рода алгоритмы позволяют оценивать эффекты от какого либо воздействия, поэтому может возникнуть задача оптимального управления, где методы оптимизации будут чрезвычайно полезны или даже необходимы для принятия решений.
В СУБД решается задача поиска оптимального вычислительного графа, целевой функцией является "сложность" вычислений, которую нужно минимизировать.

Для формулировки оптимизационной задачи в прикладной области часто требуется определенная экспертиза, чтобы сформулировать, что хотим максимизировать/минимизировать, какие условия надо соблюсти и т.п. 
Также надо понимать, а какие есть инструменты для решения задачи.
На первых порах, когда проект только зарождается нередко используется первый попавшийся солвер. Нам на практике встречались даже реализации в Excel, что для MVP-решения является нормальным подходом, но при дальнейшем масштабировании задачи, время поиска решения может сильно возрасти. Чтобы уложиться в приемлемое расчётное время, необходимо подбирать другой солвер. В этой статье мы рассмотрим несколько открытых солверов, сравним их возможности и скорость работы на модельной задаче ценообразования.


<!-- 
Тему связанную с оптимизацией объять одной статьей невозможно, так как существует очень много классов задач и, соответственно, методов их решения.
Здесь постараемся рассмотреть некоторые постановки задач оптимизации в ритейле и разберем поподробнее на практически важной задаче Ценообразования возможные постановки и сравним некоторые подходы к решению, основанные на открытых библиотеках в python
 -->


Прежде всего рассмотрим постановку задачи в общем виде:

$x$ - вектор размерностью $n$

$f(x) \to \min(\max)$ - целевая функция

$g_i(x) \leqslant 0, \ i=1..m$ - ограничения вида неравенства

$h_i(x) = 0, \ j=1..k$ - ограничения вида равенства

$x \in X$ - допустимое множество значений переменных ($ \mathbb{R}, \mathbb{Z}$ и т.п.)

Исходя из практики можно разложить данную постановку на несколько классов в зависимости от вида целевой функции, ограничений и $X$

* __Безусловная оптимизация__ $g_i(x), h_j(x)$ - отсутствуют, $X = \mathbb{R}^n$

* __LP__ (linear programming) - линейное программирование. $f(x), g_i(x), h_j(x)$ - линейные функции, $X = \mathbb{R}_+^n$. 
$\mathbb{R}_+$

* __MILP__ (mixed integer linear programming) - смешанное целочисленное линейное программирование, это задача LP в которой часть переменных являются целочисленными

* __NLP__ (nonlinear programming) - нелинейное програмирование, возникает когда хотя бы одна из функций $f(x),\ g_i(x),\ h_j(x)$ нелинейна

* __MINLP__ (mixed integer nonlinear programming) - смешанное целочисленное нелинейное программирование, по аналогии с __MILP__ в случае наличия целочисленных переменных


__NLP__ в свою очередь можно подробить еще на кучу разных классов в зависимости от вида нелинейности и выпуклости


### Примеры из области ритейла.


#### Оптимальное распределение маркетингового бюджета
Реализовать выделенный бюджет на маркетинговые активности максимально эффективно. Есть несколько каналов для рекламных акций, выделенный бюджет, цель - максимально выгодно инвестировать бюджет, чтобы суммарный доход со всем коммуникаций был максимален. Также необходимо учесть бизнес ограничения на нагрузку каждого канала + частоту взаимодействия.


#### Планирование ассортимента
Подобрать уровни ассортимента мерчгрупп так, чтобы максимизировать оборот с учетом полочного пространства выделенного на мерчгруппу.


#### Закупка товаров
Задача - распределить бюджет выделенный на закупки для поддержания товарооборота, достаточного уровня сервиса, и при этом достигать определенных финансовых показателей под выделенный бюджет на закупки


#### Ценообразование
Поиск оптимальной конфигурации цен с учетом ценового позиционирования, допустимых ценовых диапазонов для каждого товара и набора бизнес-ограничений. Цены должны максимизировать суммарный доход, а прибыль быть не ниже на заданного уровня.


## Обзор пакетов python
В указанных выше задачах возникает условная оптимизация, здесь предлагается рассмотреть полезные open-source пакеты и солверы для решения задач условной оптимизации в зависимости от ее типа, которые часто можно встретить в практических задачах.


### Открытые библиотеки, предоставляющие интерфейс для решения оптимизационных задач

__Scipy__ - библиотека, которая содержит большой набор функций для научных вычислений, в том числе имеет инструменты для решения оптимизационных задач, находящиеся в модуле scipy.optimize. В модуле находятся методы для решения задач линейного программирования, нелинейного программирования(как условная так и безусловная). 
В документации [scipy.optimize.minimize](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html) можно найти, что условную оптимизацию с нелинейнойстью поддерживают __cobyla__, __slsqp__, __trust-constr__, также там можно найти краткое описание методов и ссылки на статьи, [статья на хабре](https://habr.com/ru/company/ods/blog/448054/).

[__Pyomo__](http://www.pyomo.org/) - пакет, который содержит ряд инструментов для формулирования, решения и анализа оптимизационных моделей. Главная особенность - это удобный интерфейс для структурированного формулирования оптимизационной задачи и поддержка большого количества солверов, в том числе коммерческих. По сути pyomo занимается “перевариванием” сформулированной модели в формат, понятный для запускаемого солвера, потом забирает его и выплевывает в интерфейс python. Является частью проекта [COIN-OR](https://www.coin-or.org/), который содержит также ряд солверов. Среди них можно выделить [Ipopt](https://github.com/coin-or/Ipopt), [Cbc](https://github.com/coin-or/Cbc). Ipopt позволяет находить локальные оптимумы в задаче NLP с помощью прямо-двойственного метода внутренней точки, подробнее в оригальной [статье](http://www.optimization-online.org/DB_HTML/2004/03/836.html). Cbc - в нем реализован метод решения задачи MILP, основанный на алгоритме сочетающий в себе метод ветвей и границ и секущих плоскостей [wiki](https://en.wikipedia.org/wiki/Branch_and_cut). Также для LP имеется поддержка пакета [glpk](https://en.wikipedia.org/wiki/GNU_Linear_Programming_Kit).

[__Cvxpy__](https://www.cvxpy.org/index.html) - данный пакет специально заточен для решения задач выпуклой оптимизации (convex optimization). После того как задача сформулирована, перед решением проверяется выпуклость и аффинность целевой функции и ограничений с помощью правил [DCP](https://www.cvxpy.org/tutorial/dcp/index.html#:~:text=Disciplined%20convex%20programming%20(DCP)%20is,they%20are%20applied%20by%20CVXPY) (disciplined convex programming), после проверки задача преобразуется в стандартную форму и передается квадратичному или коническому солверу. Полный список солверов и типов задач, с помощью которых можно решить их можно найти [здесь](https://www.cvxpy.org/tutorial/advanced/index.html).

Итого можно сформировать таблицу с солверами, библиотеками и видами задач, которые позволяют решать солверы:

| Солвер(метод) | Пакеты в python | NLP | LP | MILP | MINLP |
|:-------------:|:---------------:|:---:|:--:|:----:|:-----:|
|     cobyla    |      scipy      |  y  |  n |   n  |    n  |
|     slsqp     |      scipy      |  y  |  n |   n  |    n  |
|  trust-constr |      scipy      |  y  |  n |   n  |    n  |
|     ipopt     |      pyomo      |  y  |  y |   n  |    n  |
|      glpk     |   pyomo, cvxpy  |  n  |  y |   y  |    n  |
|      cbc      |   pyomo, cvxpy  |  n  |  y |   y  |    n  |

### Модельная задача оптимизации в ценообразовании

Как уже было сказано выше, одной из основных целей ценообразования может быть формирование таких цен, которые:

* Соответствуют некоторому набору правил, бизнес-логике и т.п.

* Позволяют достигать определенных целей, например, повышение выручки, трафика, без просадки в других показателях.

Первый пункт по сути задает нам диапазон в котором производится поиск новой цены, возможные соотношения между ценами на товары, соотношения цен с ценами конкурентов, условия на (не)изменение цены.

Второй задает нам целевой функционал для задачи оптимизации и набор ограничений, которые контролируют остальные показатели.

__Рассмотрим следующую постановку задачи__:

Необходимо поднять выручку, при условии, что маржа не должна просесть, т.е. оставаться на уровне не ниже текущей в абсолютном выражении. При этом необходимо соблюдать следующие ценовые границы: $\pm 10$% от текущей цены, $\pm 15$% от среднерыночной цены, если диапазоны накладываются, то финальный диапазон - пересечение, в противном случае отклонение от рынка не рассматривается, остается только отклонение от текущей цены. Также необходимо обеспечивать равенство цен товаров, находящихся в одной линейке(это, например, товары которые отличаются только вкусом вроде йогурта). И в конце не забыть, что цена должна быть формата x.99

Чтобы начать решать задачу, требуется знать отклик изменения спроса на изменение цены, в данной статье мы не будем отдельно останавливаться на том как оценивать этот отклик, будем считать что он задан в следующем виде:

\begin{equation}
\tag{1.1}
Q(P) = Q_{0} \exp\bigg(E \cdot \bigg(\frac{P}{P_0} - 1\bigg)\bigg)
\end{equation}

, где $Q$ - спрос по новой цене $P$, $Q_0$ - спрос по текущей цене $P_{0}$ , $E$ - коэффициент эластичности. 
Для удобства введем переменую 

\begin{equation}
\tag{1.2}
x = \frac{P}{P_0},
\end{equation}

тогда функцию спроса можно переписать в следующем виде.

\begin{equation}
\tag{1.3}
Q(x) = Q_0 \exp(E \cdot (x-1))
\end{equation}

Можно сформулировать поставленную задачу как задачу __NLP__ или __LP__, рассморим оба варианта.




Введем дополнительные обозначения:

$n$ - количество товаров

$C_i$ - себестоимость товара $i$, 

$R$ - выручка, 

$M$ - фронт-маржа или для краткости будем говорить просто маржа. 

$\gamma$ - набор индексов товаров, находящихся в одной линейке, $\gamma_{j}$ - $j-$ая линейка, в которой $m_j$ товаров, $\gamma_{j, k}$ - $k$-ый товар в $j$-ой линейке

__NLP__ постановка задачи
$$
x \in \mathbb{R} ^ n
$$

\begin{equation}
\tag{2.1}
R(x) = \sum_{i=1}^{n} P_i \cdot Q_i(x_i) = \sum_{i=1}^{n} P_{0, i} \cdot Q_{0, i} \exp{(E_i\cdot(x_i-1))} \to \max_{x}
\end{equation}

\begin{equation}
\tag{2.2}
x_i \in [x_{l, i}, x_{u, i}], \ i=1..n
\end{equation}

\begin{equation}
\tag{2.3}
M(x) = \sum_{i=1}^{n} (P_i - C_i) \cdot Q_i(x_i) = \sum_{i=1}^{n} (P_{0, i} - C_i) \cdot Q_{0, i} \exp{(E_i\cdot(x_i-1))} \geqslant M_0
\end{equation}

\begin{equation}
\tag{2.4.1}
x_{\gamma_{j, 1}} = x_{\gamma_{j, 2}} = ... = x_{\gamma_{j, m_j}}, \ j=1..k
\end{equation}

Ограничение (2.4.1) также можно записать в матричной форме:

\begin{equation}
\tag{2.4.2}
A \cdot x = 0, 
\end{equation}
Все элементы матрицы нулевые, кроме
\begin{equation}
\ A(\gamma_{j,1}, \gamma_{j, 1}) = -A(\gamma_{j, 1}, \gamma_{j, 2}) = 1,\ ..\ , A(\gamma_{j,m_j-1}, \gamma_{j, m_j-1}) = -A(\gamma_{j, m_j-1}, \gamma_{j, m_j}) = 1, \ j=1..k
\end{equation}

(2.1) - это целевой функционал, (2.2) - ценовой диапазон, в котором ищется цена для каждого товара, (2.3) - ограничение на маржу снизу, (2.4.1), (2.4.2) - описывает равенство цен в линейках, здесь испульзуются $x$, так как подразумевается, что текущая цена является одинаковой во всей линейке.
В данной постановке $M_0$ - это текущее значение маржи:

\begin{equation}
\tag{2.5}
M_0 = \sum_{i=1}^{n} (P_i - C_i) \cdot Q_{0, i} 
\end{equation}


Цена, которая будет пересчитана по (1.2) является непрерывной, поэтому после необходимо округлить.

__LP__ постановка задачи

Преимуществом данного подхода является то, что цены можно сразу искать по сетке, которые удовлетворяют ограничению (2.2) и правилам округления. Например, если текущая цена 99.99, то сетка цен может быть такой $\hat{P}$ = [89.99; 90.99; ... 108.99; 109.99], всего 21 значение, новую цену можно закодировать бинарной маской длиной 21 с одной единичкой, этого можно добиться введя переменную - бинарный вектор длины 21 $\hat{x}$, у которого сумма всех элементов равна 1: $sum(\hat{x}) = 1$, тогда точно будет выбрана одна цена. Для каждой цены необходимо получить сетку значений продаж $\hat{Q}$, при этом в общем случае нет необходимости в наличии аналитической функции спроса, $\hat{Q}$ можно брать хоть из бустинга или т.п. Что касается линеек, можно было бы ввести дополнительные ограничения на равенство $\hat{x}$ по аналогии с (2.4.1), но намного проще и дешевле схлопнуть $\hat{Q}$ по линейкам, так как цены внутри них будут искаться по одной и той же сетке и поиск, соответственно, производить по линейкам

Итого, чтобы сформулировать задачу, нужно 
* сформировать сетку цен размера $g_i$ для каждого товара $\hat{P}_i$, которые удовлетворяют граничным условиям и правилам округления
* задать бинарный вектор $\hat{x}_i$ 
* посчитать уровень спроса $\hat{Q}_i$ для сетки цен $\hat{P}_i$
* схлопнуть $\hat{Q}_i$ по линейкам, взяв сумму

Обозначим за $n'$ - количество линеек

Таким образом, получим такую постановку:


\begin{equation}
\tag{3.1}
\hat{x}_i \in \{0, 1\}^{g_i},\ i=1..n'
\end{equation}

\begin{equation}
\tag{3.2}
R(\hat{x}) = \sum_{i=1}^{n'} \sum_{j=1}^{g_i} \hat{P}_{i, j} \cdot \hat{Q}_{i, j} \cdot \hat{x}_{i, j}
\end{equation}

\begin{equation}
\tag{3.3}
\sum_{j=1}^{g_i} \hat{x}_{i, j} = 1,\ i=1..n'
\end{equation}

\begin{equation}
\tag{3.4}
M(\hat{x}) = \sum_{i=1}^{n'} \sum_{j=1}^{g_i} (\hat{P}_{i, j} - C_i) \cdot \hat{Q}_{i, j} \hat{x}_{i, j} \geqslant M_0
\end{equation}



<details class="spoiler">
<summary>
        |> Python Spoiler Sample Template
</summary>
<div class="spoiler__content"><pre><code class="python">
def lol(kek):
    return 'azaza'
</code></pre><p></p></div></details>

<img src="data/Без названия.png" width="1000"/>